# hw 5 &6
# Dylan Gray

In [1]:
# natural log function
from math import log

# some formatters
number = '{:.2f}'.format; money = '${:,.2f}'.format
percent = lambda i: '{:.2f}%'.format(i*100)

# some spicy meta-programming                       
def build_functions(**kwargs):  
    for name, text in kwargs.items():
        globals()['find_'+name] = eval(
        "lambda {0}:print('{3}'{4}) or ('{1}: '+{2}({3}), {3})[num]".format(
            ','.join(k+'=False' for k in (*kwargs, 'num')),
            *(s.strip() for s in text.split(';')),
            ''.join(".replace('{0}', str({0}))".format(k) for k in kwargs)))

# a handful of functions
build_functions(
    rate    = "effective rate;        percent; (end/start)**(1/periods) -1",
    periods = "accumulatory periods;  number;  log(end/start) / log(1+rate)",
    start   = "initial investment;    money;   end/(1+rate)**periods",
    end     = "ending amount;         money;   start * (1+rate)**periods",
    EAR     = "effective annual rate; percent; (1+rate/periods)**periods -1",
)

clas

5.6) **Calculating Future Values** Assume the total cost of a college education will be \$320,000 when your child enters college in 18 years. You presently have \$67,000 to invest. What APR must you earn on your investment to cover the cost of you child's college education?

In [2]:
find_rate(start=67, end=320, periods=18)

(320/67)**(1/18) -1


'effective rate: 9.08%'

5.12) **Calculating Future Values** Your coin collection contains fifty 1952 silver dollars. If your grandparents purchased them for their face value when they were new, how much will your collection be worth when you retire in 2063, assuming they appreciate at an annual rate of 4.3 percent?

In [3]:
find_end(start=50, periods=2063-1952, rate=0.043)

50 * (1+0.043)**111


'ending amount: $5,352.15'

5.19) **Calculating Future Values** You are scheduled to receive $20,000 in two years. When you receive it, you will invest it for six more years at 7.3 percent per year. How much will you have have eight years?

In [4]:
find_end(start=20000, periods=6, rate=0.073)

20000 * (1+0.073)**6


'ending amount: $30,523.08'

5.20) **Calculating Number of Periods** You expect to receive \$15000 at graduation in two years. You plan on investing it at 9 percent until you have \$75000. How long will you wait from now?

In [5]:
find_periods(start=15000, rate=0.09, end=75000)

log(75000/15000) / log(1+0.09)


'accumulatory periods: 18.68'

Adding the two extra years, you will have to wait about 20.7 years.

6.14) **Calculating EAR** First National Bank charges 12.4 percent compounded monthly on its business loans. First United Bank charges 12.7 percent compounded semiannually. As a potential borrower, which bank would you go to for a new loan?

In [6]:
# First National
find_EAR(rate=0.124, periods=12)

(1+0.124/12)**12 -1


'effective annual rate: 13.13%'

In [7]:
# First United
find_EAR(rate=0.127, periods=2)

(1+0.127/2)**2 -1


'effective annual rate: 13.10%'

For \$100 of loans, First National charges \$13.13 in interest, while First United would only charge \$13.10.
As a potential borrower, I would go to First United for a new loan.

6.20 **Calculating Loan Payments** You want to buy a new sports coupe for \$79500, and the finance office quotes you an APR of 5.8 percent for a 60-month loan. What will your monthly payments be? What is the effective annual rate on this loan?

In [8]:
pv = 79500
rate = 0.058 / 12
periods = 60
payment = (rate * pv) / (1- (1+rate)**(-periods))
'monthly_payment: ' + money(payment)

'monthly_payment: $1,529.58'

In [9]:
find_EAR(rate=0.058, periods=12)

(1+0.058/12)**12 -1


'effective annual rate: 5.96%'

6.36 **Comparing Cash Flow Streams** You've just joined the investment banking firm of Howe, Dewey, and Cheatum. They've offered two salary options, with payments at the end of each month. Interest is 7% compounded monthly.

 - 75k per year for the next two years
 - 64k per year for the next two years, and 20k now

In [10]:
n = 24
r = 0.07 / 12
p = 75e3 / 12

# annuity formula
total = p * ((1+r)**n - 1) / r
    
money(total)

'$160,506.45'

In [11]:
n = 24
r = 0.07 / 12
p = 64e3 / 12

# annuity formula
total = p * ((1+r)**n - 1) / r

# add fv of initial bonus
total += 20e3 * (1+r)**n

money(total)

'$159,961.62'

You are better off taking the \$75,000 per year for two years.

6.38) **Growing Annuity** One year from today you will begin depositing 9 percent of your annual salary in an account that will earn 10 percent per year. Your salary is $50k this year and will increase at 3 percent per year throughout your career. How much money will you have 40 years from today?

In [12]:
total = 0

for year in range(1, 40): 
    total *= 1.1
    salary = 50e3 * (1.03**year)
    total += 0.09 * salary

money(total)

'$2,514,669.64'

6.41) **Calculating Annuity Present Values** You want to borrow \$89000 to buy a new boat. You can make monthly payments of \$1850, but no more. Assuming monthly compounding, what's the highest APR you can afford on a 60-month loan?

In [17]:
class CashFlow(list):
    def pv(self, rate):
        return sum( val / (1+rate)**n
            for n, val in enumerate(self))
    
payments = CashFlow([0]+[1850]*60)

# derivative does not change signs, so binary search works
search_precision = 3000
upper_bound = 1
lower_bound = 0
for i in range(search_precision):
    APR = (upper_bound+lower_bound)/2
    pv = payments.pv(rate=APR/12)
    
    if pv < 89e3:
        upper_bound = APR
    else:
        lower_bound = APR

print('APR of', '{:.7f}%'.format(100*APR), 'gives pv of', money(pv))

APR of 9.0580022% gives pv of $89,000.00


6.58) **Calculating Annuity Values** You are considering either leasing or buying a car. The car costs \$32,000. A lease has an initial fee of \$1200, with monthly payments of \$469 for three years. APR at 6 percent. Which option should you take? What's the breakeven selling price of the car?

In [18]:
# one payment of 1200, then 36 payments of 469, at .5% per period
lease_fv = 1200
for month in range(36):
    lease_fv *= 1.005
    lease_fv += 469

# and for the buying option
pv = 32000
rate = 0.06 / 12
periods = 36
payment = (rate * pv) / (1- (1+rate)**(-periods))

buy_fv = 0
for month in range(36):
    buy_fv *= 1.005
    buy_fv += payment

net_buy_fv = buy_fv - 19e3  # sell price

print('leasing cost fv:', money(lease_fv))
print('buying cost fv:', money(net_buy_fv))

leasing cost fv: $19,884.65
buying cost fv: $19,293.78


You would be better off buying than renting.

In [19]:
print('breakeven selling price:', money(buy_fv - lease_fv))

breakeven selling price: $18,409.13
